In [74]:
import config 

## imports key&tokens from ./etc/conda/activate.d/env_vars.sh 
# consumer_key = config.consumer_key
# consumer_secret = config.consumer_secret
# access_token = config.access_token
# access_token_secret = config.access_token_secret

#nlee
consumer_key = "a4r4DVL2wiu7r5mUE1iFY4mRI"
consumer_secret = "PWscHlly5NMRUlsuPd6bKtzXNipfxAoDQPJDijX2wlNCwgOPGw"
access_token = "958937785791991808-UkIpG4Kqmzag9fRQat2VFOVBIykPbya"
access_token_secret = "hg7VSVaGGtHyhWAUTnY37GyZe5l2g3QPOxjLfDNSSejY8"


In [166]:
import tweepy
import os
import json

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

##### INITIALIZATION #####

# init tweepy API, wait instead of failing on rate_limit
api = tweepy.API(auth, wait_on_rate_limit=True)


filesize = 1024*1024 # 10mb
numfiles = 2 #2 00 * 10mb = 2gb

keepscraping = True # continue scraping until data threshold met

filecount = 1 # counter variable
ofilename = "tweets/t-" + str(filecount) + ".txt" #create filename

# streams tweets in location l 
# l = [-122.75,36.8,-121.75,37.8] # l = san fran
l = [-117.4, 33.8, -117.5, 33.9] # l = san bernardino to n.corona

##### INITIALIZATION #####

# opens file name
ofile = open(ofilename, "w+")

class MyStreamListener(tweepy.StreamListener):
    # add new tweets to tweets list
    def on_status(self, status):
        global filecount
        global ofilename
        global keepscraping
        global numfiles
        global filesize
        
        if filecount >= numfiles:
            keepscraping = false
        # if file bigger than 10mb, close old and write new file
        if (os.stat(ofilename).st_size > filesize and filecount < numfiles):
            ofile.close()
            filecount += 1
            ofile.open(ofilename, "w+")
        
        # convert status to json 
        raw = status._json
        
        created_at = raw['created_at']
        text = raw['text']
        user = raw['user']
        geodata = raw['place']
        entities = raw['entities']
        
        data = {}
        data['tweet'] = []
        data['tweet'].append(({
            'created_at': created_at,
            'text': text,
            'user': user,
            'geodata': geodata,
            'entities': entities
        }))
        
        json.dump(data, ofile)
        ofile.write("\n")
        
        
    # prevent looping errors from locking out API key
    def on_error(self, status_code):
        if status_code == 420: #blazeit
            return False


In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = MyStreamListener())

while (keepscraping):
    # creates stream
    myStream.filter(locations=l, languages=['en'])
#     except Exception, e:
#         print "Exception occured: "
#         print e
